In [1]:
from DataIO import DataIO, City, Service, TrafficType
from SleepInference.SleepInferenceAPI import SleepInferenceAPI

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
sleep_patterns = SleepInferenceAPI.detect_sleep_patterns_city(city=City.DIJON, window=5)